In [ ]:
%pip install pandas

In [3]:
%pip install scipy

In [1]:
%pip install ipywidgets

In [5]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import os

def load_goals_from_csv(file_path):
    return pd.read_csv(file_path)

def load_reweighted_goals(file_path):
    return pd.read_csv(file_path)

def select_top_communicative_goals(weighted_goals, reweighted_goals, top_n=3):
    combined_goals = pd.merge(weighted_goals, reweighted_goals, on='goal', how='outer')
    combined_goals['final_weight'] = combined_goals['weight'].fillna(0) + combined_goals['reweight'].fillna(0)
    top_goals = combined_goals.nlargest(top_n, 'final_weight')
    return top_goals

def plot_goals(top_goals, temperament):
    plt.figure(figsize=(10, 5))
    
    max_duration = top_goals['duration'].max()
    time = np.linspace(0, max_duration, num=100)
    max_weights_at_x = np.zeros_like(time)

    for index, row in top_goals.iterrows():
        goal = row['goal']
        weight = row['final_weight']
        duration = row['duration']
        
        if duration <= 0:
            print(f"Длительность для цели '{goal}' некорректна: {duration}. Пропускаем.")
            continue
        
        weights = np.where(time <= duration, weight * (1 - time / duration), 0)
        max_weights_at_x = np.maximum(max_weights_at_x, weights)

        plt.plot(time, weights, label=goal)

    plt.plot(time, max_weights_at_x, color='black', linestyle='--', label='Максимальные значения')

    plt.title(f'Вес коммуникативных целей по времени реализации ({temperament})')
    plt.xlabel('Время реализации (минуты)')
    plt.ylabel('Вес цели')
    plt.legend()
    plt.grid()
    plt.show()

print(f"""\nВ выпадающем меню ниже предоставлен выбор из ситуаций по номерам. Соответствие ситуаций и обозначений следующее.
Situation 1: Вы узнали, что ваш друг расстроен из-за разрыва отношений.
Situation 2: На встрече друзей кто-то начинает спорить о политике.
Situation 3: На встрече с заказчиком возникло недопонимание по выполненной части проекта.
Situation 4: Ваш знакомый хочет узнать ваше мнение о его новой идее.
Situation 5: Вы заметили, что ваш друг стал менее общительным.
Situation 6: На семейном празднике возникли споры о том, что делать дальше.
Situation 7: Ваш коллега не справляется с задачами и просит помощи.
Situation 8: Вы узнали о проблемах в команде на работе, при этом ответственность за проект лежит на всех участниках команды, а срок сдачи близок.
Situation 9: Ваш знакомый делится с вами своими переживаниями о будущем.
Situation 10: Ваш близкий человек говорит, что у него или у нее финансовые проблемы.
Situation 11: У вас спросили: "Как дела?".
Situation 12: Вы указываете коллеге, что он ошибся и сделал не то, что требовалось. Он в ответ возмущается, что я его критикую.
Situation 13: В вашей группе кто-то сделал проект лучше и раньше всех.
Situation 14: Вас попросили объяснить теорию вероятностей.""")

# Создание интерфейса
situation_dropdown = widgets.Dropdown(
    options=['Situation 1', 'Situation 2', 'Situation 3', 'Situation 4', 'Situation 5', 'Situation 6', 'Situation 7', 'Situation 8', 'Situation 9', 'Situation 10', 'Situation 11', 'Situation 12', 'Situation 13', 'Situation 14'], 
    description='Ситуация:',
)

load_button = widgets.Button(description='Выбрать цели')
output = widgets.Output()

# Обработчик события нажатия кнопки
def on_button_click(b):
    with output:
        clear_output()
        user_situation = situation_dropdown.value
        
        # Определяем путь к папке с данными в зависимости от выбранной ситуации
        data_folder = f'data/{user_situation}'
        
        # Загружаем взвешенные цели
        weighted_goals_path = os.path.join(data_folder, 'weighted_goals.csv')
        weighted_goals = load_goals_from_csv(weighted_goals_path)

        # Перечень темпераментов для отображения графиков
        temperaments = ['choleric', 'phlegmatic', 'sanguine', 'melancholic', 'without']

        for temperament in temperaments:
            # Загрузка перевзвешенных целей в зависимости от темперамента
            reweighted_goals_path = os.path.join(data_folder, f'reweighted_goals_{temperament}.csv')
            reweighted_goals = load_reweighted_goals(reweighted_goals_path)

            # Выбор трех коммуникативных целей
            selected_goals = select_top_communicative_goals(weighted_goals, reweighted_goals)
            
            print(f"\nВыбранные коммуникативные цели для реализации ({temperament}):")
            for goal in selected_goals['goal']:
                print(f"- {goal}")
            
            # Построение графика для выбранных целей
            plot_goals(selected_goals, temperament)

# Привязка обработчика к кнопке
load_button.on_click(on_button_click)

# Отображение элементов интерфейса
display(situation_dropdown, load_button, output)



В выпадающем меню ниже предоставлен выбор из ситуаций по номерам. Соответствие ситуаций и обозначений следующее.
Situation 1: Вы узнали, что ваш друг расстроен из-за разрыва отношений.
Situation 2: На встрече друзей кто-то начинает спорить о политике.
Situation 3: На встрече с заказчиком возникло недопонимание по выполненной части проекта.
Situation 4: Ваш знакомый хочет узнать ваше мнение о его новой идее.
Situation 5: Вы заметили, что ваш друг стал менее общительным.
Situation 6: На семейном празднике возникли споры о том, что делать дальше.
Situation 7: Ваш коллега не справляется с задачами и просит помощи.
Situation 8: Вы узнали о проблемах в команде на работе, при этом ответственность за проект лежит на всех участниках команды, а срок сдачи близок.
Situation 9: Ваш знакомый делится с вами своими переживаниями о будущем.
Situation 10: Ваш близкий человек говорит, что у него или у нее финансовые проблемы.
Situation 11: У вас спросили: "Как дела?".
Situation 12: Вы указываете коллег

Dropdown(description='Ситуация:', options=('Situation 1', 'Situation 2', 'Situation 3', 'Situation 4', 'Situat…

Button(description='Выбрать цели', style=ButtonStyle())

Output()